# Units

yt has a robust symbolic unit system for use with computations. Units can be converted into others with equivalent dimensionality, and unit arithmetic is automatically carried through. 

As before, import yt. We'll also need to import NumPy.

In [ ]:
import yt
import numpy as np

Let's also load up a sample dataset for trying things out on:

In [ ]:
# Athena++ datasets don't have unit information built into them,
# so we have to tell them what units they are in 
units_override = {"length_unit": (1.0, "kpc"), "mass_unit": (1.0e14, "Msun"), "time_unit": (1.0, "Myr")}
ds = yt.load("AM06/AM06.out1.00500.athdf", units_override=units_override)

## Basic Information about Units and Unitful Arrays and Quantities

Unitful objects in yt are the `YTArray`, which is a NumPy array with unit information, and the `YTQuantity`, which is a scalar with units. We've seen `YTArray`s already, when we return fields from data objects:

In [ ]:
sp = ds.sphere("c", (200.0, "kpc"))
rho = sp['density'].copy() # Make a copies so we can manipulate them
v_x = sp['velocity_x'].copy()
T = sp['temperature'].copy()

`YTArrays` consist of a NumPy array and units:

In [ ]:
print(rho.value) # This returns a copy of the NumPy array
print(rho.view()) # This returns the actual NumPy array
print(rho.units) # This returns the unit object

Units also have a dimensions attribute:

In [ ]:
print(rho.units.dimensions)

`YTQuantities` consist of a single floating-point value and units:

In [ ]:
one_dens = rho[0]
print(one_dens.value)
print(one_dens.units)

`YTArrays` and `YTQuantities` can also be created from scratch using the `YTArray` and `YTQuantity` classes:

In [ ]:
from yt import YTArray, YTQuantity
a = YTArray(np.random.random(10), "kpc")
print(a)

In [ ]:
b = YTQuantity(1.0e-27, "erg/s/cm**2/steradian")
print(b)

### Unit Conversions

The thing you will be doing most often with units is making conversions. We can use the `in_units()` method to make conversions of an array or quantity into another one with different units, so long as they are in the same dimensions:

In [ ]:
print(rho.in_units('g/cm**3'))
print()
print(rho.in_units('Msun/kpc**3'))
print()
print(rho.in_units('lbm/ft**3'))

`in_units()` returns a brand-new array with the new units. You can do an in-place conversion of an array with `convert_to_units()`:

In [ ]:
v_x.convert_to_units('km/s')
print(v_x)
v_x.convert_to_units('mile/hr')
print(v_x)
v_x.convert_to_units('kpc/Myr')
print(v_x)

However, trying to convert a unitful object into units of different dimensions (with `in_units` or `convert_to_units`) will have the (predictable) result:

In [ ]:
T.convert_to_units('kpc') # oops

### Arithmetic Operations with `YTArrays` and `YTQuantities`

Unitful objects with the same dimensions can be added and subtracted:

In [ ]:
rho + YTQuantity(200.0, "Msun/kpc**3")

In [ ]:
v_x - YTArray(np.ones(v_x.size), "Mpc/kyr")

Multiplication and division will carry out the appropriate unit arithmetic:

In [ ]:
rho*v_x

In [ ]:
v_x/T

We can take powers of units, even fractional ones!

In [ ]:
T**0.25

Most NumPy mathematical functions will handle units appropriately, either by passing them through, doing the appropriate arithmetic, or removing them (in the case of operations like `np.log`):

In [ ]:
np.sqrt(rho)

In [ ]:
np.abs(v_x)

In [ ]:
np.log(T)

However, once again, don't do something silly!

In [ ]:
rho + v_x

### Code Units

Each dataset also has a set of "code units" associated with it, which are the "default" units of the dataset, and the units in which the on-disk fields are returned. 

In [ ]:
for unit in ["length", "mass", "time", "velocity", "magnetic"]:
    print("code_%s =" % unit, getattr(ds, unit+"_unit"))

We can convert to and from these units all the same:

In [ ]:
v_x.in_units("code_length/code_time")

You can also create arbitary arrays and quantities with code units, but you will need to use the methods `ds.arr()` and `ds.quan()` to do so from a particular dataset:

In [ ]:
v = ds.arr([1.0, 2.0, 3.0], "code_length/code_time")
print(v.in_units("km/s"))

In [ ]:
p = ds.quan(3.0e-3, "code_mass/code_time**2/code_length")
print(p.in_units("keV/cm**3"))

### Physical Constants

yt maintains a database of physical constants with units. They are `YTQuantities`! You can import them from `yt.units`, and use them in computations:

In [ ]:
from yt.units import G, kboltz, hbar
print(G)
print()
print(kboltz)
print()
print(hbar)

In [ ]:
print(kboltz*T)

## Unit Systems

The default set of units provided by yt are "Gaussian" cgs units, expressing all calculations in terms of centimeters, grams, seconds, Kelvin, radians, and derived units in terms of these. This means that most fields are returned in either these units, though some fields are returned in code units and some derived fields have special units.

As seen above, it is possible to convert these units into those from other systems, but yt also provides methods for converting arrays, quantities, and the fields for entire datasets into alternative unit systems.

First of all, simple methods are provided for conversion into MKSA and cgs units:

In [ ]:
# Methods for MKSA and cgs/Gaussian units
print(rho.in_mks())
print()
print(rho.in_cgs())

However, yt has an entire set of unit systems that can be converted to. Let's list them:

In [ ]:
list(yt.unit_system_registry.keys())

Let's look at the default units of a few of these unit systems:

In [ ]:
yt.unit_system_registry['cgs']

In [ ]:
yt.unit_system_registry['galactic']

In [ ]:
yt.unit_system_registry['imperial']

In [ ]:
yt.unit_system_registry['geometrized']

We can use the `in_base()` method to convert any `YTArray` or `YTQuantity` into any particular base:

In [ ]:
print(rho.in_base('imperial'))
print()
print(rho.in_base('galactic'))
print()
print(rho.in_base('geometrized'))

By default, yt returns fields of datasets in cgs units. You can specify a unit system in the call to `yt.load()` to return fields in alternative unit systems:

In [ ]:
ds_mks = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030", unit_system="mks")
ds_mks.r[:,:,:]["velocity_z"]

In [ ]:
ds_imp = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030", unit_system="imperial")
ds_imp.r[:,:,:]["velocity_z"]

In [ ]:
ds_gal = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030", unit_system="galactic")
ds_gal.r[:,:,:]["velocity_z"]

## Equivalencies

Some quantities are not directly convertible to one another in yt (they don't have the same dimensions) but nevertheless are related via some physical relation or law. For this, yt provides a set of unit equivalencies.

* `"thermal"`: conversions between temperature and energy ($E = k_BT$)
* `"spectral"`: conversions between wavelength, frequency, and energy for photons ($E = h\nu = hc/\lambda, c = \lambda\nu$)
* `"mass_energy"`: conversions between mass and energy ($E = mc^2$)
* `"lorentz"`: conversions between velocity and Lorentz factor ($\gamma = 1/\sqrt{1-(v/c)^2}$)
* `"schwarzschild"`: conversions between mass and Schwarzschild radius ($R_S = 2GM/c^2$)
* `"compton"`: conversions between mass and Compton wavelength ($\lambda = h/mc$)

The following unit equivalencies only apply under conditions applicable for an ideal gas with a constant mean molecular weight $\mu$ and ratio of specific heats $\gamma$:

* `"number_density"`: conversions between density and number density ($n = \rho/\mu{m_p}$)
* `"sound_speed"`: conversions between temperature and sound speed for an ideal gas ($c_s^2 = \gamma{k_BT}/\mu{m_p}$)

In [ ]:
T.to_equivalent("keV", "thermal")

In [ ]:
rho.to_equivalent("cm**-3", "number_density")

In [ ]:
from yt.units import solar_mass
solar_mass.to_equivalent("mile", "schwarzschild")